###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import sys
sys.path.append("../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
tofullcensus = geodict.tofullcensus
geotogeoid = geodict.geotogeoid
import sqlite3 as sq

# This notebook outlines the download and formatting process for data from the Tennessee Bureau of Investigation.  

Go to this page: https://crimeinsight.tbi.tn.gov/public/View/dispview.aspx?ReportId=20, "Crime Rates by Jurisdiction" fom this page: https://crimeinsight.tbi.tn.gov/public/Browse/browsetables.aspx
We are looking for Crime Rates by county and the State of Tennessee here. Select "Jurisdiction by Geography" on the left-side of the screen, where you are then able to manipulate geographies by level. Select the State, and then deselect the regions, select counties in the GNRC operating area (most in Middle, some in West), and under those counties leave the jurisdictions. Click "show report" in the top right-side of the screen, and then download a multi-dimensional csv file. Delete the data headers, rename Jurisdiction by Geography to "NAME", make sure that the numbers are not formatted with commas, add a county column to tag the juridictions with a county, leaving the column null for the state and counties, and set the column headers correctly before saving the file as "TBRCrimeRatesbyJurisdiction_IncidentDate(YYYY).csv.  

Next, From the homepage select "Crime and Clearance by Offense Type", deselect clearance in the left-side selector. Select "Jurisdiction by Geography" and similarly select the State and all Counties and jurisdictions. Download this, format, transpose data so NAME is on the left and you can create a similar county column as before, and save as 

*Note*: the jurisdictions don't necessarily repond to a place level plan in all cases, but they may and you can toy around here to see if they match the plan or document you're generating data for!


## Crime Type  
Right now I only have this for 2021 and 2022, keeping years separate haven't needed them together yet

In [3]:
#import and examine
data = pd.read_csv('../Data Downloads/TBI_CrimeTypebyJurisdiction_IncidentDate2023.csv')
data['County'] = data['County'].fillna('Full')
data.head(3)

,NAME,County,All Offense Types,Crimes Against Persons,Murder,Negligent Manslaughter,Justifiable Homicide,Negligent Vehicular Manslaughter,Kidnapping/Abduction,Forcible Rape,Forcible Sodomy,Sexual Assault W/Object,Forcible Fondling,Incest,Statutory Rape,Aggravated Assault,Simple Assault,Intimidation,Stalking,Commercial Sex Acts,Involuntary Servitude,Crimes Against Property,Arson,Bribery,Burglary,Counterfeiting/Forgery,Destruction/Damage/Vandalism,Embezzlement,Extortion/Blackmail,Fraud - False Pretenses,Fraud - Credit Card/ATM,Fraud - Impersonation,Fraud - Welfare,Fraud - Wire,Fraud - Identity Theft,Fraud - Computer Hacking/Invasion,Robbery,Theft - Pocket-picking,Theft - Purse Snatching,Theft - Shoplifting,Theft From Building,Theft From Coin Machine,Theft From Motor Vehicle,Theft of Motor Vehicle Parts,Theft - All Other Larceny,Motor Vehicle Theft,Stolen Property Offenses,Crimes Against Society,Animal Cruelty,Drug/Narcotic Violations,Drug/Narcotic Equipment Violations,Gambling - Betting/Wagering,Gambling - Operating/Promoting,Gambling - Equipment Violations,Gambling - Sports Tampering,Pornography/Obscene Material,Prostitution,Prostitution Assisting/Promoting,Purchasing Prostitution,Weapon Law Violations,Missing,All Offense Types.1,Crimes Against Persons.1,Murder.1,Negligent Manslaughter.1,Justifiable Homicide.1,Negligent Vehicular Manslaughter.1,Kidnapping/Abduction.1,Forcible Rape.1,Forcible Sodomy.1,Sexual Assault W/Object.1,Forcible Fondling.1,Incest.1,Statutory Rape.1,Aggravated Assault.1,Simple Assault.1,Intimidation.1,Stalking.1,Commercial Sex Acts.1,Involuntary Servitude.1,Crimes Against Property.1,Arson.1,Bribery.1,Burglary.1,Counterfeiting/Forgery.1,Destruction/Damage/Vandalism.1,Embezzlement.1,Extortion/Blackmail.1,Fraud - False Pretenses.1,Fraud - Credit Card/ATM.1,Fraud - Impersonation.1,Fraud - Welfare.1,Fraud - Wire.1,Fraud - Identity Theft.1,Fraud - Computer Hacking/Invasion.1,Robbery.1,Theft - Pocket-picking.1,Theft - Purse Snatching.1,Theft - Shoplifting.1,Theft From Building.1,Theft From Coin Machine.1,Theft From Motor Vehicle.1,Theft of Motor Vehicle Parts.1,Theft - All Other Larceny.1,Motor Vehicle Theft.1,Stolen Property Offenses.1,Crimes Against Society.1,Animal Cruelty.1,Drug/Narcotic Violations.1,Drug/Narcotic Equipment Violations.1,Gambling - Betting/Wagering.1,Gambling - Operating/Promoting.1,Gambling - Equipment Violations.1,Gambling - Sports Tampering.1,Pornography/Obscene Material.1,Prostitution.1,Prostitution Assisting/Promoting.1,Purchasing Prostitution.1,Weapon Law Violations.1,Missing.1
0,Tennessee,Full,512485.0,149921.0,714.0,44.0,NaN,24.0,2115.0,2217.0,483.0,210.0,2599.0,28.0,376.0,37131.0,74449.0,27424.0,2013.0,89.0,5.0,265408.0,918.0,21.0,21325.0,5058.0,47884.0,2051.0,1027.0,14181.0,7093.0,4024.0,112.0,2182.0,489.0,214.0,5170.0,698.0,168.0,33156.0,14471.0,199.0,33620.0,8195.0,28073.0,32935.0,2144.0,97156.0,677.0,49677.0,30041.0,26.0,69.0,16.0,NaN,1181.0,260.0,43.0,53.0,15113.0,NaN,418713.0,119001.0,665.0,40.0,59.0,21.0,1801.0,2180.0,461.0,206.0,2386.0,27.0,373.0,25714.0,61645.0,23800.0,1860.0,80.0,5.0,251964.0,918.0,21.0,21325.0,5058.0,47884.0,2051.0,1027.0,14181.0,7093.0,4024.0,112.0,2182.0,489.0,214.0,5170.0,698.0,168.0,33156.0,14471.0,199.0,33620.0,8195.0,28073.0,32372.0,2144.0,69241.0,677.0,49677.0,30041.0,26.0,69.0,16.0,NaN,1181.0,260.0,43.0,53.0,15113.0,NaN
1,Cheatham,Full,1900.0,609.0,3.0,NaN,NaN,NaN,12.0,4.0,1.0,1.0,11.0,NaN,1.0,150.0,287.0,128.0,11.0,NaN,NaN,714.0,2.0,NaN,49.0,42.0,135.0,4.0,1.0,53.0,24.0,34.0,2.0,23.0,NaN,NaN,1.0,NaN,NaN,61.0,33.0,NaN,44.0,23.0,100.0,60.0,23.0,577.0,3.0,328.0,189.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,55.0,NaN,1419.0,442.0,3.0,NaN,NaN,NaN,10.0,4.0,1.0,1.0,10.0,NaN,1.0,94.0,225.0,97.0,10.0,NaN,NaN,657.0,2.0,NaN,49.0,42.0,135.0,4.0,1.0,53.0,24.0,34.0,2.0,23.0,NaN,NaN,1.0,NaN,NaN,61.0,33.0,NaN,44.0,23.0,100.0,60.0,23.0,415.0,3.0,328.0,189.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,55.0,NaN
2,Cheatham County Sheriff's Office,Cheatham,1284.0,480.0,3.0,NaN,NaN,NaN,11.0

In [4]:
#create a dictionary to rename the counties to full census names
countydict = {'Cheatham': 'Cheatham County, Tennessee', 'Davidson': 'Davidson County, Tennessee', 'Maury': 'Maury County, Tennessee', 
              'Robertson': 'Robertson County, Tennessee', 'Rutherford': 'Rutherford County, Tennessee', 'Sumner': 'Sumner County, Tennessee', 
              'Trousdale': 'Trousdale County, Tennessee', 'Williamson': 'Williamson County, Tennessee', 'Wilson': 'Wilson County, Tennessee', 
              'Dickson': 'Dickson County, Tennessee', 'Houston': 'Houston County, Tennessee', "Humphreys": 'Humphreys County, Tennessee',
              'Montgomery': 'Montgomery County, Tennessee', 'Stewart': 'Stewart County, Tennessee'}

In [5]:
data = data.set_index('County').transpose()
data = data.rename(columns = countydict)
data = data.transpose().reset_index()

In [6]:
aside = data.loc[data['County'] != 'Full']
data = data.loc[data['County'] == 'Full'].drop(columns = 'County')

In [7]:
data.head()

,NAME,All Offense Types,Crimes Against Persons,Murder,Negligent Manslaughter,Justifiable Homicide,Negligent Vehicular Manslaughter,Kidnapping/Abduction,Forcible Rape,Forcible Sodomy,Sexual Assault W/Object,Forcible Fondling,Incest,Statutory Rape,Aggravated Assault,Simple Assault,Intimidation,Stalking,Commercial Sex Acts,Involuntary Servitude,Crimes Against Property,Arson,Bribery,Burglary,Counterfeiting/Forgery,Destruction/Damage/Vandalism,Embezzlement,Extortion/Blackmail,Fraud - False Pretenses,Fraud - Credit Card/ATM,Fraud - Impersonation,Fraud - Welfare,Fraud - Wire,Fraud - Identity Theft,Fraud - Computer Hacking/Invasion,Robbery,Theft - Pocket-picking,Theft - Purse Snatching,Theft - Shoplifting,Theft From Building,Theft From Coin Machine,Theft From Motor Vehicle,Theft of Motor Vehicle Parts,Theft - All Other Larceny,Motor Vehicle Theft,Stolen Property Offenses,Crimes Against Society,Animal Cruelty,Drug/Narcotic Violations,Drug/Narcotic Equipment Violations,Gambling - Betting/Wagering,Gambling - Operating/Promoting,Gambling - Equipment Violations,Gambling - Sports Tampering,Pornography/Obscene Material,Prostitution,Prostitution Assisting/Promoting,Purchasing Prostitution,Weapon Law Violations,Missing,All Offense Types.1,Crimes Against Persons.1,Murder.1,Negligent Manslaughter.1,Justifiable Homicide.1,Negligent Vehicular Manslaughter.1,Kidnapping/Abduction.1,Forcible Rape.1,Forcible Sodomy.1,Sexual Assault W/Object.1,Forcible Fondling.1,Incest.1,Statutory Rape.1,Aggravated Assault.1,Simple Assault.1,Intimidation.1,Stalking.1,Commercial Sex Acts.1,Involuntary Servitude.1,Crimes Against Property.1,Arson.1,Bribery.1,Burglary.1,Counterfeiting/Forgery.1,Destruction/Damage/Vandalism.1,Embezzlement.1,Extortion/Blackmail.1,Fraud - False Pretenses.1,Fraud - Credit Card/ATM.1,Fraud - Impersonation.1,Fraud - Welfare.1,Fraud - Wire.1,Fraud - Identity Theft.1,Fraud - Computer Hacking/Invasion.1,Robbery.1,Theft - Pocket-picking.1,Theft - Purse Snatching.1,Theft - Shoplifting.1,Theft From Building.1,Theft From Coin Machine.1,Theft From Motor Vehicle.1,Theft of Motor Vehicle Parts.1,Theft - All Other Larceny.1,Motor Vehicle Theft.1,Stolen Property Offenses.1,Crimes Against Society.1,Animal Cruelty.1,Drug/Narcotic Violations.1,Drug/Narcotic Equipment Violations.1,Gambling - Betting/Wagering.1,Gambling - Operating/Promoting.1,Gambling - Equipment Violations.1,Gambling - Sports Tampering.1,Pornography/Obscene Material.1,Prostitution.1,Prostitution Assisting/Promoting.1,Purchasing Prostitution.1,Weapon Law Violations.1,Missing.1
0,Tennessee,512485.0,149921.0,714.0,44.0,NaN,24.0,2115.0,2217.0,483.0,210.0,2599.0,28.0,376.0,37131.0,74449.0,27424.0,2013.0,89.0,5.0,265408.0,918.0,21.0,21325.0,5058.0,47884.0,2051.0,1027.0,14181.0,7093.0,4024.0,112.0,2182.0,489.0,214.0,5170.0,698.0,168.0,33156.0,14471.0,199.0,33620.0,8195.0,28073.0,32935.0,2144.0,97156.0,677.0,49677.0,30041.0,26.0,69.0,16.0,NaN,1181.0,260.0,43.0,53.0,15113.0,NaN,418713.0,119001.0,665.0,40.0,59.0,21.0,1801.0,2180.0,461.0,206.0,2386.0,27.0,373.0,25714.0,61645.0,23800.0,1860.0,80.0,5.0,251964.0,918.0,21.0,21325.0,5058.0,47884.0,2051.0,1027.0,14181.0,7093.0,4024.0,112.0,2182.0,489.0,214.0,5170.0,698.0,168.0,33156.0,14471.0,199.0,33620.0,8195.0,28073.0,32372.0,2144.0,69241.0,677.0,49677.0,30041.0,26.0,69.0,16.0,NaN,1181.0,260.0,43.0,53.0,15113.0,NaN
1,Cheatham,1900.0,609.0,3.0,NaN,NaN,NaN,12.0,4.0,1.0,1.0,11.0,NaN,1.0,150.0,287.0,128.0,11.0,NaN,NaN,714.0,2.0,NaN,49.0,42.0,135.0,4.0,1.0,53.0,24.0,34.0,2.0,23.0,NaN,NaN,1.0,NaN,NaN,61.0,33.0,NaN,44.0,23.0,100.0,60.0,23.0,577.0,3.0,328.0,189.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,55.0,NaN,1419.0,442.0,3.0,NaN,NaN,NaN,10.0,4.0,1.0,1.0,10.0,NaN,1.0,94.0,225.0,97.0,10.0,NaN,NaN,657.0,2.0,NaN,49.0,42.0,135.0,4.0,1.0,53.0,24.0,34.0,2.0,23.0,NaN,NaN,1.0,NaN,NaN,61.0,33.0,NaN,44.0,23.0,100.0,60.0,23.0,415.0,3.0,328.0,189.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,55.0,NaN
6,Davidson,86460.0,24115.0,104.0,6.0,NaN,NaN,253.0,331.0,134.0,82.0,402.0,2.0,42.0,6659.0,1114

In [8]:
#transpose to aggregate regions, then transpose back
data = data.set_index('NAME').transpose()
data = data.rename(columns = countydict)
GNRCCounties = [data['Stewart County, Tennessee'],data['Montgomery County, Tennessee'],
                data['Houston County, Tennessee'],data['Humphreys County, Tennessee'],
                data['Dickson County, Tennessee'],data['Cheatham County, Tennessee'],
                data['Robertson County, Tennessee'],data['Sumner County, Tennessee'],
                data['Davidson County, Tennessee'],data['Wilson County, Tennessee'],
                data['Trousdale County, Tennessee'],data['Williamson County, Tennessee'],
                data['Rutherford County, Tennessee']]
data['GNRC'] = sum(GNRCCounties)
GNRCCountiesAll = [data['Stewart County, Tennessee'],data['Montgomery County, Tennessee'],
                   data['Houston County, Tennessee'],data['Humphreys County, Tennessee'],
                   data['Dickson County, Tennessee'],data['Cheatham County, Tennessee'],
                   data['Robertson County, Tennessee'],data['Sumner County, Tennessee'],
                   data['Davidson County, Tennessee'],data['Wilson County, Tennessee'],
                   data['Trousdale County, Tennessee'],data['Williamson County, Tennessee'],
                   data['Rutherford County, Tennessee'],data['Maury County, Tennessee']]
data['GNRC Region'] = sum(GNRCCountiesAll)
MPOCounties = [data['Robertson County, Tennessee'],data['Sumner County, Tennessee'],
               data['Davidson County, Tennessee'],data['Wilson County, Tennessee'],
               data['Williamson County, Tennessee'],data['Rutherford County, Tennessee'],
               data['Maury County, Tennessee']]
data['MPO'] = sum(MPOCounties)
data = data.transpose().reset_index()

In [9]:
#make the county column the same as the name for the full counties.. and state whatever
data['County'] = data['NAME']
data = pd.concat([data, aside])

In [10]:
#set the index to name and county to fill the null values as 0 for mathematical operations
data = data.set_index(['NAME', 'County'])

In [11]:
data.fillna(0, inplace = True)

In [12]:
#aggregate crime type groups then drop the originals
thelist = [data['Negligent Manslaughter'], data['Negligent Vehicular Manslaughter']]
data['Manslaughter'] = sum(thelist)
thelist = [data['Forcible Rape'], data['Forcible Sodomy'], data['Sexual Assault W/Object'], data['Forcible Fondling'], data['Incest'], data['Statutory Rape'], 
          data['Aggravated Assault'], data['Simple Assault']]
data['Assault & Violent Sex Crime'] = sum(thelist)
thelist = [data['Intimidation'], data['Stalking']]
data['Intimidation & Stalking'] = sum(thelist)
thelist = [data['Arson'], data['Destruction/Damage/Vandalism']]
data['Arson, Destruction & Vandalism'] = sum(thelist)
thelist = [data['Fraud - False Pretenses'], data['Fraud - Credit Card/ATM'], data['Fraud - Impersonation'], data['Fraud - Welfare'], data['Fraud - Wire'], 
          data['Fraud - Identity Theft'], data['Fraud - Computer Hacking/Invasion']]
data['Fraud'] = sum(thelist)
thelist = [data['Theft - Pocket-picking'], data['Theft - Purse Snatching'], data['Theft - Shoplifting'], data['Theft From Building'], 
           data['Theft From Coin Machine'], data['Theft From Motor Vehicle'], data['Theft of Motor Vehicle Parts'], data['Theft - All Other Larceny'], 
           data['Motor Vehicle Theft'], data['Stolen Property Offenses']]
data['Theft'] = sum(thelist)
thelist = [data['Drug/Narcotic Violations'], data['Drug/Narcotic Equipment Violations']]
data['Drugs & Narcotics Violations'] = sum(thelist)
thelist = [data['Bribery'], data['Counterfeiting/Forgery'], data['Embezzlement'], data['Extortion/Blackmail'],
           data['Gambling - Betting/Wagering'], data['Gambling - Operating/Promoting'], data['Gambling - Equipment Violations'], 
           data['Gambling - Sports Tampering']]
data['Bribery, Forgery, Blackmail, Embezzlement & Gambling'] = sum(thelist)
thelist = [data['Prostitution'], data['Prostitution Assisting/Promoting'], data['Purchasing Prostitution'], 
           data['Commercial Sex Acts'], data['Pornography/Obscene Material']]
data['Prostitution, Commercial Sex Acts & Pornography, Aiding & Purchasing Prostitution'] = sum(thelist)
cols = ['Negligent Manslaughter','Negligent Vehicular Manslaughter','Forcible Rape','Forcible Sodomy','Sexual Assault W/Object','Forcible Fondling','Incest',
        'Statutory Rape','Aggravated Assault','Simple Assault','Intimidation','Stalking','Arson','Destruction/Damage/Vandalism','Bribery','Counterfeiting/Forgery',
        'Embezzlement','Extortion/Blackmail','Fraud - False Pretenses','Fraud - Credit Card/ATM','Fraud - Impersonation','Fraud - Welfare','Fraud - Wire',
        'Fraud - Identity Theft','Fraud - Computer Hacking/Invasion', 'Theft - Pocket-picking','Theft - Purse Snatching','Theft - Shoplifting','Theft From Building',
        'Theft From Coin Machine','Theft From Motor Vehicle','Theft of Motor Vehicle Parts','Theft - All Other Larceny','Motor Vehicle Theft',
        'Commercial Sex Acts', 'Pornography/Obscene Material',
        'Stolen Property Offenses','Drug/Narcotic Violations','Drug/Narcotic Equipment Violations','Gambling - Betting/Wagering','Gambling - Operating/Promoting',
        'Gambling - Equipment Violations','Gambling - Sports Tampering','Prostitution','Prostitution Assisting/Promoting','Purchasing Prostitution', 
        'Involuntary Servitude']# there is just hardly any of this I am dropping it for this analysis
data = data.drop(columns = cols)

C:\Users\jmccall\AppData\Local\Temp\ipykernel_10668\1944912294.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['Manslaughter'] = sum(thelist)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_10668\1944912294.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['Assault & Violent Sex Crime'] = sum(thelist)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_10668\1944912294.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  C

In [13]:
#reset the index and examine
data = data.reset_index()

In [14]:
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'Tennessee Bureau of Investigation'

In [15]:
#final check
data.head()

,NAME,County,All Offense Types,Crimes Against Persons,Murder,Justifiable Homicide,Kidnapping/Abduction,Crimes Against Property,Burglary,Robbery,Crimes Against Society,Animal Cruelty,Weapon Law Violations,Missing,All Offense Types.1,Crimes Against Persons.1,Murder.1,Negligent Manslaughter.1,Justifiable Homicide.1,Negligent Vehicular Manslaughter.1,Kidnapping/Abduction.1,Forcible Rape.1,Forcible Sodomy.1,Sexual Assault W/Object.1,Forcible Fondling.1,Incest.1,Statutory Rape.1,Aggravated Assault.1,Simple Assault.1,Intimidation.1,Stalking.1,Commercial Sex Acts.1,Involuntary Servitude.1,Crimes Against Property.1,Arson.1,Bribery.1,Burglary.1,Counterfeiting/Forgery.1,Destruction/Damage/Vandalism.1,Embezzlement.1,Extortion/Blackmail.1,Fraud - False Pretenses.1,Fraud - Credit Card/ATM.1,Fraud - Impersonation.1,Fraud - Welfare.1,Fraud - Wire.1,Fraud - Identity Theft.1,Fraud - Computer Hacking/Invasion.1,Robbery.1,Theft - Pocket-picking.1,Theft - Purse Snatching.1,Theft - Shoplifting.1,Theft From Building.1,Theft From Coin Machine.1,Theft From Motor Vehicle.1,Theft of Motor Vehicle Parts.1,Theft - All Other Larceny.1,Motor Vehicle Theft.1,Stolen Property Offenses.1,Crimes Against Society.1,Animal Cruelty.1,Drug/Narcotic Violations.1,Drug/Narcotic Equipment Violations.1,Gambling - Betting/Wagering.1,Gambling - Operating/Promoting.1,Gambling - Equipment Violations.1,Gambling - Sports Tampering.1,Pornography/Obscene Material.1,Prostitution.1,Prostitution Assisting/Promoting.1,Purchasing Prostitution.1,Weapon Law Violations.1,Missing.1,Manslaughter,Assault & Violent Sex Crime,Intimidation & Stalking,"Arson, Destruction & Vandalism",Fraud,Theft,Drugs & Narcotics Violations,"Bribery, Forgery, Blackmail, Embezzlement & Gambling","Prostitution, Commercial Sex Acts & Pornography, Aiding & Purchasing Prostitution",GEO_ID,Source
0,Tennessee,Tennessee,512485.0,149921.0,714.0,0,2115.0,265408.0,21325.0,5170.0,97156.0,677.0,15113.0,0,418713.0,119001.0,665.0,40.0,59.0,21.0,1801.0,2180.0,461.0,206.0,2386.0,27.0,373.0,25714.0,61645.0,23800.0,1860.0,80.0,5.0,251964.0,918.0,21.0,21325.0,5058.0,47884.0,2051.0,1027.0,14181.0,7093.0,4024.0,112.0,2182.0,489.0,214.0,5170.0,698.0,168.0,33156.0,14471.0,199.0,33620.0,8195.0,28073.0,32372.0,2144.0,69241.0,677.0,49677.0,30041.0,26.0,69.0,16.0,0,1181.0,260.0,43.0,53.0,15113.0,0,68.0,117493.0,29437.0,48802.0,28295.0,153659.0,79718.0,8268.0,1626.0,0400000US47,Tennessee Bureau of Investigation
1,"Cheatham County, Tennessee","Cheatham County, Tennessee",1900.0,609.0,3.0,0,12.0,714.0,49.0,1.0,577.0,3.0,55.0,0,1419.0,442.0,3.0,0.0,0.0,0.0,10.0,4.0,1.0,1.0,10.0,0.0,1.0,94.0,225.0,97.0,10.0,0.0,0.0,657.0,2.0,0.0,49.0,42.0,135.0,4.0,1.0,53.0,24.0,34.0,2.0,23.0,0.0,0.0,1.0,0.0,0.0,61.0,33.0,0.0,44.0,23.0,100.0,60.0,23.0,415.0,3.0,328.0,189.0,0.0,0.0,0.0,0,2.0,0.0,0.0,0.0,55.0,0,0.0,455.0,139.0,137.0,136.0,344.0,517.0,47.0,2.0,0500000US47021,Tennessee Bureau of Investigation
2,"Davidson County, Tennessee","Davidson County, Tennessee",86460.0,24115.0,104.0,0,253.0,47408.0,2953.0,1176.0,14937.0,35.0,4489.0,0,70944.0,19982.0,96.0,5.0,17.0,0.0,231.0,328.0,134.0,82.0,385.0,2.0,42.0,5018.0,9647.0,4295.0,193.0,3.0,2.0,45095.0,84.0,2.0,2953.0,515.0,7144.0,427.0,279.0,2148.0,1785.0,815.0,8.0,448.0,5.0,16.0,1176.0,485.0,29.0,5067.0,3051.0,36.0,9603.0,1753.0,3849.0,5629.0,90.0,11633.0,35.0,6720.0,3538.0,3.0,1.0,7.0,0,131.0,7.0,3.0,3.0,4489.0,0,6.0,18800.0,4947.0,7228.0,5225.0,29603.0,10258.0,1234.0,147.0,0500000US47037,Tennessee Bureau of Investigation
3,"Maury County, Tennessee","Maury County, Tennessee",7873.0,2911.0,4.0,0,24.0,3520.0,265.0,27.0,1442.0,20.0,120.0,0,6322.0,2259.0,4.0,0.0,0.0,0.0,22.0,23.0,10.0,0.0,53.0,0.0,7.0,315.0,1457.0,390.0,27.0,0.0,0.0,3299.0,7.0,0.0,265.0,89.0,632.0,36.0,35.0,429.0,140.0,156.0,1.0,49.0,7.0,1.0,27.0,3.0,0.0,469.0,204.0,4.0,315.0,47.0,379.0,196.0,26.0,1036.0,20.0,834.0,450.0,0.0,0.0,0.0,0,17.0,1.0,0.0,0.0,120.0,0,0.0,2385.0,498.0,639.0,783.0,1646.0,1284.0,160.0,18.0,0500000US47119,Tennessee Bureau of 

In [17]:
data['Year'] = '2023'

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 85 columns):
 #   Column                                                                             Non-Null Count  Dtype  
---  ------                                                                             --------------  -----  
 0   NAME                                                                               141 non-null    object 
 1   County                                                                             141 non-null    object 
 2   All Offense Types                                                                  141 non-null    float64
 3   Crimes Against Persons                                                             141 non-null    float64
 4   Murder                                                                             141 non-null    float64
 5   Justifiable Homicide                                                               141 non-null    int64  

In [19]:
#export to the SQLite database
conn = sq.connect('../Outputs/TBI.db')
data.to_sql('TBI_CrimeType_2023', conn, if_exists = 'replace', index = False)

141